In [2]:
#imports
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
sys.path.append('..')
import scipy as sp

from extremevalue.gpd import GEVMLE_ts

import pymc as pm
import pymc_experimental.distributions as pmx
import arviz as az

In [4]:
# https://discourse.pymc.io/t/pymc-sampling-error-latent-gp-for-gev-distribution/11798

X_ = np.load("../X_.npy")
ys_maxima = np.load("../ys_maxima.npy")

In [9]:
with pm.Model() as model:
    ell = pm.InverseGamma('ell', alpha=2, beta=1, shape=(3, 20))
    # eta = pm.HalfNormal('eta', sigma=1, shape=3)

    cov_shape = pm.gp.cov.Exponential(input_dim=20, ls=ell[0])
    cov_loc = pm.gp.cov.Exponential(input_dim=20, ls=ell[1])
    cov_scale = pm.gp.cov.Exponential(input_dim=20, ls=ell[2])

    gp_shape = pm.gp.Latent(cov_func=cov_shape)
    gp_loc = pm.gp.Latent(cov_func=cov_loc)
    gp_log_scale = pm.gp.Latent(cov_func=cov_scale)

    xi = gp_shape.prior('xi', X=X_)
    mu = gp_loc.prior('mu', X=X_)
    log_sigma = gp_log_scale.prior('log_sigma', X=X_)
    # sigma = pm.Deterministic('sigma', pm.math.exp(log_sigma))
    
    yy = pmx.GenExtreme('yy', mu=mu, sigma=pm.math.exp(log_sigma), xi=0.25*pm.math.tanh(xi), observed=ys_maxima.T.ravel())
    trace = pm.sample(1000, target_accept=0.98, random_seed=10)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


SamplingError: Initial evaluation of model at starting point failed!
Starting values:
{'ell_log__': array([[ 0.01571654, -0.34856599, -0.49873131, -0.57287988,  0.95534123,
        -0.1932971 , -0.26328508, -0.31880178,  0.58296286,  0.24081685,
        -0.48594691, -0.04651921,  0.83193068,  0.59385794,  0.94022596,
        -0.37847663,  0.80851476, -0.44475941,  0.21921031,  0.46220179],
       [-0.46190823, -0.78741816,  0.26251951, -0.73848071,  0.21674675,
        -0.0866746 , -0.41586096,  0.6267628 , -0.53237638,  0.3950809 ,
         0.05028009,  0.49221925, -0.06302158, -0.9886447 ,  0.94108383,
        -0.16683282, -0.71656123, -0.45427157, -0.25311339, -0.36919614],
       [-0.62788554, -0.81593876, -0.40796008,  0.67111069, -0.63890045,
        -0.82365475, -0.29924511,  0.44730359,  0.9544388 , -0.35422697,
        -0.09957119, -0.25878895,  0.4054046 ,  0.99217339, -0.94764923,
        -0.53809181,  0.26204534, -0.97314574,  0.60726928, -0.22251816]]), 'xi_rotated_': arra